In [ ]:
from scipy.stats import norm, genextreme

In [ ]:
import numpy as np
import pandas as pd

from astropy.table import Table
from astropy import units
from astropy.units import imperial
from astropy.visualization import quantity_support

import ipywidgets as ipw

from climate_explore import ClimateData, TempData

quantity_support()

%matplotlib widget

from matplotlib import pyplot as plt

In [ ]:
all_data = Table.read("psci378-extremes.csv")

In [ ]:
c = ClimateData(all_data)

In [ ]:
pad_me = ipw.Layout(padding="5px")
locations = ipw.Dropdown(options=c.locations, default=None)
extreme_types_drop = ipw.ToggleButtons(
    options=[
        ("High", "TMAX"),
        ("Low", "TMIN"),
        #("Max precipitation", "PRCP")
    ]
)
extreme_types_label = ipw.Label("Extreme type")
extreme_types = ipw.VBox(children=[extreme_types_label, extreme_types_drop])
unit_selector = ipw.ToggleButtons(description="Units", options=[("metric", units.Celsius), ("english", units.imperial.Fahrenheit)], layout=pad_me) # ipw.Dropdown(options=["metric", "english"])

center = ipw.VBox()
left = ipw.VBox()
left.children = [extreme_types, unit_selector]

#plot_selector = ipw.Tab()

app = ipw.AppLayout(center=center, left_sidebar=left, header=locations, pane_heights=[1,10,0])

#locations.observe(update_location,

In [ ]:
def get_extreme_type_name():
    return extreme_types_drop.options[extreme_types_drop.index][0]

In [ ]:
with plt.ioff():
    fig, ax = plt.subplots()
    lines = ax.plot([1800, 2025], [50, 50])
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    low_year = ax.plot([xmin, xmin], [ymin, ymax], "-", alpha=0.7, color="green")
    high_year = ax.plot([xmax, xmax], [ymin, ymax], "-", alpha=0.7, color="green")
    trend = ax.plot([1800, 2025], [50, 50], alpha=0, color="orange")
    trend = trend[0]
    ax.set_xlabel("Year")
    ax.set_ylabel(f"AVERAGE {get_extreme_type_name()} Temperature")
    ax.grid()


In [ ]:
year_range = ipw.IntRangeSlider(min=c.years.min(), max=c.years.max(), step=1, description="Select years", layout=dict(width="90%"))
fig.canvas.layout = ipw.Layout(width="90%")
year_range.value = (year_range.min, year_range.max)
tabs = ipw.Tab()

range_and_average = ipw.VBox()

range_and_average.children = [year_range, fig.canvas]
tabs.children = [range_and_average]
tabs.titles = ("1) Select years",)
center.children = [tabs]

In [ ]:
selected_location_data = ""

In [ ]:
def update_trend_line():
    bobber = selected_location_data
    trend_fit = bobber.fit_trend(group_by="YEAR")
    trend.set_data(bobber.data['YEAR'], trend_fit(bobber.data['YEAR']))
    return bobber

In [ ]:
def make_subtitle():
    return ""

In [ ]:
def update_graph_line(dummy=None):
    selected_location_data.start_year, selected_location_data.end_year = year_range.value
    selected_location_data.extreme_type = get_extreme_type_name().lower()
    selected_location_data.display_unit = unit_selector.value
    years, temps = selected_location_data.max_min_temps_by_year(group_by='YEAR')
    lines[0].set_data(years, temps)
    ax.set_ylim((np.nanmin(temps) - 10 * temps.unit), (np.nanmax(temps) + 10 * temps.unit))
    ax.set_xlim(years.min() - 5, years.max() + 5)
    ax.set_ylabel(f"AVERAGE {get_extreme_type_name()} Temperature ({unit_selector.value:latex})")
    subset = update_trend_line()
    rate_change = subset.fit_trend(group_by="YEAR").coef[1] * subset.display_unit * 100
    rate_change_string = rate_change.to_string(format="latex", precision=3)
    ax.set_title(f"{c.location} \nchange in 100 years {rate_change_string}")
    update_year_markers({})

In [ ]:
def update_location(change):
    global selected_location_data
    c.set_location(locations.value)
    selected_location_data = TempData(data=c.selected_data,
                                      display_unit=unit_selector.value,
                                      extreme_type=get_extreme_type_name().lower()
                                     )

    ax.set_title(f"{c.location}")
    update_graph_line()
    year_range.min, year_range.max = selected_location_data.year_range

    year_range.value = (year_range.min, year_range.max)
    trend.set_alpha(1)
    fig.canvas.draw()
    fig.canvas.flush_events()

In [ ]:
def update_year_markers(change):
    xmin, xmax = year_range.value
    ymin, ymax = ax.get_ylim()
    high_year[0].set_data([xmax, xmax], [ymin, ymax])
    low_year[0].set_data([xmin, xmin], [ymin, ymax])
    selected_location_data.start_year, selected_location_data.end_year = year_range.value
    subset = update_trend_line()
    rate_change = subset.fit_trend(group_by="YEAR").coef[1] * subset.display_unit * 100
    ax.set_title(f"{c.location} \nchange in 100Y {rate_change:.3f}")
    fig.canvas.draw()
    fig.canvas.flush_events()


In [ ]:
def update_temp_unit(change):
    update_graph_line()

In [ ]:
locations.observe(update_location, names="value")

In [ ]:
year_range.observe(update_year_markers, names="value")
unit_selector.observe(update_temp_unit, names="value")
extreme_types_drop.observe(update_graph_line, names="value")

In [ ]:
locations.value = "FARGO HECTOR INTERNATIONAL AIRPORT, ND US"

In [ ]:
app